# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [7]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [8]:
def expected_return(P, rf, mkt_rp):
    expected = 0
    invested = P['Amount Invested'].sum()
    
    for i in range(len(P)):
        expected += (P.loc[i, 'Amount Invested'] / invested) * (rf + (P.loc[i, 'Beta'] * mkt_rp))
        
    return expected

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [9]:
def volatility(P, cor):
    total_vol = 0
    invested = P['Amount Invested'].sum()
    
    for i in range(len(P)):
        amt = P.loc[i, 'Amount Invested'] / invested
        stock_vol = P.loc[i, 'Volatility']
        total_vol += pow(amt, 2) * pow(stock_vol, 2)
        
    extra = len(P) * cor
    
    for i in range(len(P)):
        extra *= P.loc[i, 'Volatility'] * (P.loc[i, 'Amount Invested'] / invested)
    vol += extra
    vol = np.sqrt(vol)
    
    return vol

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [10]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    x = vol/mkt_vol
    expected = rf + (x * mkt_rp)
    
    return expected

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [2]:
def CAPM_beta(return_path, factors_path):
    df = pd.read_csv(return_path)
    
    df['month'] = df['month'].astype(str).str.zfill(2)
    df['year'] = df['year'].astype(str)
    df['date'] = df['year'] + df['month']
    df['month'] = df['month'].astype(int)
    df['year'] = df['year'].astype(int)
    df['date'] = df['date'].astype(int)
    
    first_df = pd.read_csv(factors_path)
    
    merge_df = pd.merge(df, first_df, on='date')
    merge_df['Mkt-RF'] = merge_df['Mkt-RF']/100
    merge_df['RF'] = merge_df['RF']/100
    
    expected = merge_df.groupby('CUSIP')['RET'].mean().reset_index(name='EXP_RETURN')
    
    merge_df['excess_return_firm'] = merge_df['RET'] - merge_df['RF']
    merge_df['excess_return_market'] = merge_df['Mkt-RF']

    groups = merge_df.groupby('CUSIP')
    beta = groups.apply(lambda x: np.cov(x['excess_return_firm'], x['excess_return_market'])[0, 1] / 
                                      x['excess_return_market'].var()).reset_index(name='BETA')
    
    final_df = pd.merge(expected, beta, on='CUSIP')
    
    output = pd.DataFrame(data = {
        "CUSIP" : final_df['CUSIP'].values,
        "EXP_RETURN" : final_df['EXP_RETURN'].values,
        "BETA" : final_df['BETA'].values,
    })
    
    return output

return_path = '../../datafiles/A7/monthlycrsp_mod7.csv'
factors_path = '../../datafiles/A7/factors.CSV'

# Takes a while (~10 min)
data = CAPM_beta(return_path, factors_path)
data

CUSIP  EXP_RETURN       BETA
0  36020   -0.032023  -0.483703
1  36021    0.028259  -1.124363
2  36022    0.048830  -5.149374
3  36023    0.089881  10.187062

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [12]:
def slope(data):
    curve = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    
    return curve[1]